In [1]:
from datetime import datetime, timedelta
from backtesting import Backtest, Strategy
import pandas as pd
import ta
import numpy as np
from binance.client import Client
import ccxt
import mplfinance as mpf
import numpy as np
from math import floor
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import config

client = Client(config.api_key, config.api_secret)
# Create Binance client instance
# exchange = ccxt.binance({
#     'apiKey': config.api_key,
#     'secret': config.api_secret,
# })

C:\Users\islom\PycharmProjects\Python-Binance\venv\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [55]:
def get_candle_1m_after(time):
    start = str(time + timedelta(minutes=1))
    end = str(time + timedelta(hours=10))
    df = pd.DataFrame(client.get_historical_klines(symbol="BNBUSDT", interval='1m', start_str=start, end_str=end))
    df = df.iloc[:, :5]
    df.columns = ['time', 'open', 'high', 'low', 'close']
    df = df.astype(float)
    # df = df.set_index('time')
    df.time = pd.to_datetime(df.time, unit='ms')
    # df = df.astype(float)
    return df

def get_candle_1h_before(time):
    start = str(time - timedelta(hours=5))
    end = str(time)
    df = pd.DataFrame(client.get_historical_klines(symbol="BNBUSDT", interval='1h', start_str=start, end_str=end))
    df = df.iloc[:, :5]
    df.columns = ['time', 'open', 'high', 'low', 'close']
    df = df.set_index('time')
    df.index = pd.to_datetime(df.index, unit='ms')
    df = df.astype(float)
    return df

In [3]:
def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean()
    exp2 = price.ewm(span = slow, adjust = False).mean()
    macd = pd.DataFrame(exp1 - exp2).rename(columns = {'close':'macd'})
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
    frames =  [macd, signal, hist]
    df = pd.concat(frames, join = 'inner', axis = 1)
    return df

In [5]:
def get_data(symbol, interval='1h'):
    df = pd.DataFrame(client.get_historical_klines(symbol=symbol, interval=interval))
    df = df.iloc[:, :5]
    df.columns = ['time', 'open', 'high', 'low', 'close']
    df = df.astype(float)
    df.time = pd.to_datetime(df.time, unit='ms')
    mac = get_macd(df['close'], 26, 12, 9)
    ret = pd.concat([df, mac], join = 'inner', axis = 1)
    return ret

In [13]:
def get_signal(data):
    buy_price = []
    macd_signal = []

    for i in range(len(data)):
        try:
            if data['macd'][i-1] < data['signal'][i-1]:
                if data['macd'][i] > data['signal'][i]:
                    buy_price.append(data.iloc[i].close)
                    macd_signal.append(1)
                else:
                    buy_price.append(np.nan)
                    macd_signal.append(0)
            else:
                buy_price.append(np.nan)
                macd_signal.append(0)
        except:
                buy_price.append(np.nan)
                macd_signal.append(0)
    return pd.DataFrame({"buy_price": buy_price, "macd_signal": macd_signal})

In [18]:
df = get_data('BNBUSDT')
signal = get_signal(df)
all = pd.concat([df, signal], join = 'inner', axis = 1)

In [19]:
all[all['macd_signal'] ==1]

,time,open,high,low,close,macd,signal,hist,buy_price,macd_signal
14,2023-03-22 05:00:00,337.3,337.5,336.9,337.2,-0.467474,-0.550328,0.082854,337.2,1
37,2023-03-23 04:00:00,320.6,322.0,320.4,320.5,-3.515405,-3.517648,0.002243,320.5,1
79,2023-03-24 23:00:00,321.8,322.5,321.5,322.1,-1.346292,-1.381538,0.035246,322.1,1
105,2023-03-26 01:00:00,323.0,323.9,322.7,323.7,-0.231774,-0.310415,0.078641,323.7,1
157,2023-03-28 05:00:00,308.6,310.3,308.5,310.2,-4.033397,-4.239386,0.205989,310.2,1
208,2023-03-30 08:00:00,315.5,317.3,315.1,316.7,-0.059911,-0.118003,0.058092,316.7,1
220,2023-03-30 20:00:00,316.1,317.4,315.6,317.1,0.294395,0.262933,0.031463,317.1,1
237,2023-03-31 13:00:00,316.8,319.3,316.5,318.2,0.311432,0.202443,0.108989,318.2,1
250,2023-04-01 02:00:00,317.6,319.0,316.6,317.2,0.248255,0.244895,0.003360,317.2,1
268,2023-04-01 20:00:00,314.5,315.5,314.3,314.4,-0.511767,-0.526573,0.014806,314.4,1


312.62

In [67]:
profit = pd.DataFrame(columns=['buy_price', 'profit', 'price'], index=range(36))
n = 0
for x in range(len(all)):
    if all.iloc[x].macd_signal == 1:
        row1 = all.iloc[x]
        profit.iloc[n]['buy_price'] = row1.close
        after = get_candle_1m_after(df.iloc[x].time)
        for y in range(len(after)):
            row2 = after.iloc[y]
            if row1.close <= row2.close - (row2.close * 0.01):
                profit.iloc[n]['profit'] = 0
                profit.iloc[n]['price'] = row2.close
                n += 1
                break
            elif row1.close >= row2.close + (row2.close * 0.015):
                profit.iloc[n]['profit'] = 1
                profit.iloc[n]['price'] = row2.close
                n += 1
                break
        else:
            profit.iloc[n]['profit'] = 2
            n += 1

In [68]:
profit

,buy_price,profit,price
0,337.2,1,331.9
1,320.5,0,324.1
2,322.1,2,NaN
3,323.7,2,NaN
4,310.2,2,NaN
5,316.7,2,NaN
6,317.1,2,NaN
7,318.2,2,NaN
8,317.2,2,NaN
9,314.4,2,NaN


In [58]:
len(profit)

36

In [54]:
for x in range(10):
    if x == 5:
       print(x)
    elif x == 3:
        break
else:
    print(x)

In [119]:
result = pd.concat([df, macd, signal], axis=1)
result = result[['timestamp', 'open', 'high', 'low', 'close', 'volume', 'macd', 'signal', 'hist', 'buy_price', 'sell_price', 'macd_signal']]
result = result.iloc[:, 1:]
result.timestamp = pd.to_datetime(result.timestamp)
result.iloc[:10 : -1]

,timestamp,open,high,low,close,volume,macd,signal,hist,buy_price,sell_price,macd_signal
999,2023-04-30 11:00:00,322.2,322.4,321.6,321.7,3014.644,-0.337669,-0.463755,0.126086,NaN,NaN,0
998,2023-04-30 10:00:00,322.1,322.6,321.7,322.2,6509.091,-0.343461,-0.495277,0.151816,NaN,NaN,0
997,2023-04-30 09:00:00,322.1,322.3,321.7,322.1,4611.320,-0.396768,-0.533230,0.136463,NaN,NaN,0
996,2023-04-30 08:00:00,321.5,322.4,321.5,322.0,4693.970,-0.448853,-0.567346,0.118493,NaN,NaN,0
995,2023-04-30 07:00:00,322.1,322.4,321.5,321.6,5910.864,-0.498623,-0.596969,0.098347,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15,2023-03-20 10:00:00,337.3,338.7,336.5,337.3,19655.626,-0.798160,-1.019320,0.221160,NaN,NaN,0
14,2023-03-20 09:00:00,337.6,338.2,335.9,337.4,20436.360,-0.961057,-1.074610,0.113553,337.4,NaN,1
13,2023-03-20 08:00:00,334.7,338.2,334.2,337.6,39552.105,-1.165175,-1.102998,-0.062177,NaN,NaN,0
12,2023-03-20 07:00:00,333.5,335.8,333.5,334.7,42475.501,-1.429120,-1.087454,-0.341666,NaN,NaN,0


In [120]:
signals = result[result['macd_signal'] == 1]
signals.iloc[: :-1]

,timestamp,open,high,low,close,volume,macd,signal,hist,buy_price,sell_price,macd_signal
993,2023-04-30 05:00:00,321.9,322.3,321.7,322.3,4087.009,-0.577437,-0.648403,0.070966,322.3,NaN,1
957,2023-04-28 17:00:00,322.6,322.7,321.8,322.3,9173.150,-2.582274,-2.626530,0.044256,322.3,NaN,1
931,2023-04-27 15:00:00,330.4,333.7,329.3,331.8,39271.450,-1.423026,-1.446827,0.023801,331.8,NaN,1
904,2023-04-26 12:00:00,338.2,345.0,338.1,344.6,76243.163,2.101270,1.813964,0.287306,344.6,NaN,1
883,2023-04-25 15:00:00,330.3,332.7,329.8,332.4,28711.593,0.036703,-0.063019,0.099722,332.4,NaN,1
869,2023-04-25 01:00:00,331.1,332.4,331.1,331.6,15561.030,0.310334,0.308192,0.002142,331.6,NaN,1
863,2023-04-24 19:00:00,331.7,332.2,331.5,332.0,10528.637,0.309169,0.264865,0.044304,332.0,NaN,1
856,2023-04-24 12:00:00,331.2,332.5,331.1,332.3,21116.984,0.291411,0.269337,0.022075,332.3,NaN,1
844,2023-04-24 00:00:00,330.6,332.7,329.4,332.4,27088.801,0.429767,0.393934,0.035833,332.4,NaN,1
801,2023-04-22 05:00:00,323.4,324.1,323.1,323.7,10961.245,-0.321082,-0.396489,0.075407,323.7,NaN,1
